<a href="https://colab.research.google.com/github/soodeh-nilforoushan/NLP-Transformers/blob/main/7_POS_Tagger.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers datasets

In [2]:
from datasets import load_dataset
import nltk
from nltk.corpus import brown
import json

import torch
from sklearn.metrics import f1_score,accuracy_score, confusion_matrix
from sklearn.model_selection import train_test_split

In [3]:
nltk.download('brown')
nltk.download('universal_tagset')

[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data]   Package brown is already up-to-date!
[nltk_data] Downloading package universal_tagset to /root/nltk_data...
[nltk_data]   Package universal_tagset is already up-to-date!


True

In [4]:
corpus=brown.tagged_sents(tagset="universal")
corpus

[[('The', 'DET'), ('Fulton', 'NOUN'), ('County', 'NOUN'), ('Grand', 'ADJ'), ('Jury', 'NOUN'), ('said', 'VERB'), ('Friday', 'NOUN'), ('an', 'DET'), ('investigation', 'NOUN'), ('of', 'ADP'), ("Atlanta's", 'NOUN'), ('recent', 'ADJ'), ('primary', 'NOUN'), ('election', 'NOUN'), ('produced', 'VERB'), ('``', '.'), ('no', 'DET'), ('evidence', 'NOUN'), ("''", '.'), ('that', 'ADP'), ('any', 'DET'), ('irregularities', 'NOUN'), ('took', 'VERB'), ('place', 'NOUN'), ('.', '.')], [('The', 'DET'), ('jury', 'NOUN'), ('further', 'ADV'), ('said', 'VERB'), ('in', 'ADP'), ('term-end', 'NOUN'), ('presentments', 'NOUN'), ('that', 'ADP'), ('the', 'DET'), ('City', 'NOUN'), ('Executive', 'ADJ'), ('Committee', 'NOUN'), (',', '.'), ('which', 'DET'), ('had', 'VERB'), ('over-all', 'ADJ'), ('charge', 'NOUN'), ('of', 'ADP'), ('the', 'DET'), ('election', 'NOUN'), (',', '.'), ('``', '.'), ('deserves', 'VERB'), ('the', 'DET'), ('praise', 'NOUN'), ('and', 'CONJ'), ('thanks', 'NOUN'), ('of', 'ADP'), ('the', 'DET'), ('City

In [5]:
# for sent in corpus:
#   brown_json=json.dumps(sent)
corpus_dict={}
words=[]
pos_tags=[]

####Different way
# for sentence_tag_pairs in corpus:
#   tokens=[]
#   target=[]
#   for token, tag in sentence_tag_pairs:
#     words.append(token)
#     pos_tags.append(tag)

for i,sentence in enumerate(corpus):
  word=[]
  pos_tag=[]
  for j,tuple in enumerate(corpus[i]):
    word.append(corpus[i][j][0])
    pos_tag.append(corpus[i][j][1])
  words.append(word)
  pos_tags.append(pos_tag)


import json
with open("data.json","w") as f:
  for x,y in zip(words,pos_tags):
    corpus_dict={'words':x, 'pos_tags':y}
    # print(corpus_dict)
    s=json.dumps(corpus_dict)
    f.write(f"{s}")

In [6]:
from datasets import load_dataset
data=load_dataset("json",data_files="data.json")

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [7]:
#because dataset is too large and it takes time, we only use 20000 samples
dataset = data["train"].shuffle(seed=42).select(range(20_000))
split_data= dataset.train_test_split(test_size=0.3,seed=42)
split_data

DatasetDict({
    train: Dataset({
        features: ['words', 'pos_tags'],
        num_rows: 14000
    })
    test: Dataset({
        features: ['words', 'pos_tags'],
        num_rows: 6000
    })
})

In [8]:
split_data["train"][0]

{'words': ['The',
  'need',
  'for',
  'interchange',
  'and',
  'understanding',
  'makes',
  'vital',
  'the',
  'full',
  'use',
  'of',
  'all',
  'methods',
  'of',
  'consultation',
  '.'],
 'pos_tags': ['DET',
  'NOUN',
  'ADP',
  'NOUN',
  'CONJ',
  'VERB',
  'VERB',
  'ADJ',
  'DET',
  'ADJ',
  'NOUN',
  'ADP',
  'PRT',
  'NOUN',
  'ADP',
  'NOUN',
  '.']}

In [10]:
t=tokenizer(split_data["train"][0]["words"],is_split_into_words=True)
t

NameError: ignored

In [14]:
t.tokens()

['[CLS]',
 'The',
 'need',
 'for',
 'interchange',
 'and',
 'understanding',
 'makes',
 'vital',
 'the',
 'full',
 'use',
 'of',
 'all',
 'methods',
 'of',
 'consultation',
 '.',
 '[SEP]']

In [15]:
t.word_ids()

[None, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, None]

In [11]:
#set is a uniqe list that why we use it
#map targets to ints
target=set()
for t in pos_tags:
  target=target.union(t)
target

{'.',
 'ADJ',
 'ADP',
 'ADV',
 'CONJ',
 'DET',
 'NOUN',
 'NUM',
 'PRON',
 'PRT',
 'VERB',
 'X'}

In [12]:
#we convert target it to list since it has  ordering
target_list=list(target)
#order target
id2label={k:v for k,v in enumerate(target_list)}
label2id={v:k for k,v in id2label.items()}
print(label2id)
print(id2label)

{'NUM': 0, 'ADJ': 1, 'PRON': 2, '.': 3, 'DET': 4, 'ADV': 5, 'ADP': 6, 'X': 7, 'NOUN': 8, 'PRT': 9, 'VERB': 10, 'CONJ': 11}
{0: 'NUM', 1: 'ADJ', 2: 'PRON', 3: '.', 4: 'DET', 5: 'ADV', 6: 'ADP', 7: 'X', 8: 'NOUN', 9: 'PRT', 10: 'VERB', 11: 'CONJ'}


In [13]:
from transformers import AutoTokenizer
#tokenize dataset
checkpoint="distilbert-base-cased"
tokenizer=AutoTokenizer.from_pretrained(checkpoint)

In [14]:
t=tokenizer(split_data["train"][3]["words"],is_split_into_words=True)

In [15]:
t.tokens()

['[CLS]',
 'Blanche',
 '##d',
 'p',
 '##eanut',
 '##s',
 ',',
 'as',
 'prepared',
 'for',
 'making',
 'p',
 '##eanut',
 'butter',
 'or',
 'for',
 'eating',
 'as',
 'nuts',
 ',',
 'are',
 'r',
 '##oast',
 '##ed',
 'seeds',
 'whose',
 'seed',
 '##coat',
 '##s',
 'have',
 'been',
 'rubbed',
 'off',
 '.',
 '[SEP]']

In [16]:
#word id of the tokenized sentence
t.word_ids()

[None,
 0,
 0,
 1,
 1,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 16,
 16,
 17,
 18,
 19,
 19,
 19,
 20,
 21,
 22,
 23,
 24,
 None]

In [17]:
def align_target(labels,word_ids):
  #words id has the same length as tokenized input
  #labels is the original labels and has the same length as untokenized input
  aligned_labels=[] # it will be the same as tokenized input
  last_word=None
  for word in word_ids:
    if word is None:
      #it is a token like [CLS]
      label=-100 # this is the value the library uses to know to ignor
    else:
      #it is a real word
      #becaus NN can only work with numbers we convert it to integer thst we define above
      label=label2id[labels[word]]

    #add the label
    aligned_labels.append(label)

  return aligned_labels

In [27]:
#test the function
labels=split_data["train"][3]["pos_tags"]
word_ids=t.word_ids()
aligned_target=align_target(labels,word_ids)
aligned_target

[-100,
 10,
 10,
 8,
 8,
 8,
 3,
 6,
 10,
 6,
 10,
 8,
 8,
 8,
 11,
 6,
 10,
 6,
 8,
 3,
 10,
 10,
 10,
 10,
 8,
 4,
 8,
 8,
 8,
 10,
 10,
 10,
 9,
 3,
 -100]

In [29]:
aligned_labels=[id2label[t] if t>=0 else None for t in aligned_target]
for x,y in zip(t.tokens(),aligned_labels):
  print(f"{x}\t{y}")

[CLS]	None
Blanche	VERB
##d	VERB
p	NOUN
##eanut	NOUN
##s	NOUN
,	.
as	ADP
prepared	VERB
for	ADP
making	VERB
p	NOUN
##eanut	NOUN
butter	NOUN
or	CONJ
for	ADP
eating	VERB
as	ADP
nuts	NOUN
,	.
are	VERB
r	VERB
##oast	VERB
##ed	VERB
seeds	NOUN
whose	DET
seed	NOUN
##coat	NOUN
##s	NOUN
have	VERB
been	VERB
rubbed	VERB
off	PRT
.	.
[SEP]	None


In [30]:
#it is not only tokenizing each input sentence in the batch but also it aligns the label
def tokenize_fn(batch):
  #tokenize the input sequence first
  #this populates input_ids, attention mask and etc.
  tokenized_inputs=tokenizer(
      batch["words"],truncation=True, is_split_into_words=True
  )

  labels_batch=batch['pos_tags'] #original targets
  aligned_labels_batch=[] #store align lables
  for i , labels in enumerate(labels_batch):
    word_ids=tokenized_inputs.word_ids(i)
    aligned_labels_batch.append(align_target(labels,word_ids))

  #the 'target' must be store in key called 'labels'
  tokenized_inputs['pos_tags'] =aligned_labels_batch

  return tokenized_inputs

In [32]:
data["train"].column_names

['words', 'pos_tags']

In [34]:
tokenized_dataset=split_data.map(tokenize_fn, batched=True,remove_columns=data["train"].column_names)

In [21]:
tokenized_dataset

DatasetDict({
    train: Dataset({
        features: ['pos_tags', 'input_ids', 'attention_mask'],
        num_rows: 14000
    })
    test: Dataset({
        features: ['pos_tags', 'input_ids', 'attention_mask'],
        num_rows: 6000
    })
})

In [22]:
def flatten(list_of_lists):
  flattened=[val for sublist in list_of_lists for val in sublist]
  return flattened

In [23]:
import numpy as np
from sklearn.metrics import f1_score, accuracy_score

def compute_metrics(logits_and_labels):
  logits,labels=logits_and_labels
  preds=np.argmax(logits,axis=-1)

  #remove -100 from labels and predictions
  labels_jagged=[[t for t in label if t!= -100] for label in labels]

  #do the same predictions whenever true label is -100
  preds_jagged=[[p for p,t in zip(ps,ts) if t!= -100] \
                for ps,ts in zip (preds,labels)
                ]
  #flatten labels and preds
  labels_flat = flatten(labels_jagged)
  preds_flat = flatten(preds_jagged)

  acc= accuracy_score(labels_flat,preds_flat)
  f1=f1_score(labels_flat,preds_flat,average="macro")

  return {
      "f1": f1,
      "accuracy": acc,
  }


In [24]:
labels=[[-100,0,0,1,2,1,-100]]
logits=np.array([[
    [0.8,0.1,0,1],
    [0.8,0.1,0,1],
    [0.8,0.1,0,1],
    [0.1,0.8,0,1],
    [0.1,0.8,0,1],
    [0.1,0.8,0,1],
    [0.1,0.8,0,1],
]])
compute_metrics((logits,labels))

{'f1': 0.0, 'accuracy': 0.0}

In [35]:
from transformers import AutoModelForTokenClassification
checkpoint="distilbert-base-cased"
model=AutoModelForTokenClassification.from_pretrained(
    checkpoint,
    id2label=id2label,
    label2id=label2id,
)

Some weights of the model checkpoint at distilbert-base-cased were not used when initializing DistilBertForTokenClassification: ['vocab_transform.weight', 'vocab_layer_norm.weight', 'vocab_layer_norm.bias', 'vocab_projector.weight', 'vocab_transform.bias', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForTokenClassification were not initialized from the model checkpoint at distilbert-base-cased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this 

In [52]:
!pip install datasets transformers==4.28.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 17.4 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.32.0
    Uninstalling transformers-4.32.0:
      Successfully uninstalled transformers-4.32.0


In [36]:
from transformers import TrainingArguments

training_args=TrainingArguments(
    "distilbert-finetuned-ner",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    num_train_epochs=3,
    weight_decay=0.01,
)

In [38]:
from transformers import DataCollatorForTokenClassification
data_collator= DataCollatorForTokenClassification(tokenizer=tokenizer)

In [40]:
from transformers import Trainer
#we are using data_collator, since the hugging face library does not use the correct version by default

trainer=Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=tokenizer,
)
trainer.train()

ValueError: ignored

In [ ]:
trainer.save_model("my_saved_model")

In [ ]:
from transformers import pipeline
pipe=pipeline(
    "token-classification",
    model="my_saved_model",
    aggregation_startegy="simple",
    device=0,
)

OSError: ignored

In [ ]:
s="Bill gates was the CEO of Microsoft in Washangotn DC"
pipe(s)